In [339]:
import matplotlib
%matplotlib qt
from matplotlib.patches import Circle
import scipy.sparse as sp
import scipy.sparse.linalg as spl
import numpy as np
import matplotlib.pylab as plt

In [340]:
dx = dy = 0.01
dt = 0.1
D = 100
k = 1000
x = np.arange(0,1,dx)
y = np.arange(0,1,dy)
X,Y = np.meshgrid(x,y)

In [341]:
nx = x.size
ny = y.size
coeffs = [dx/dy*D, dy/dx*D, 0, dy/dx*D, dx/dy*D]
coeffs[2] = -sum(coeffs) - k*dx*dy
offsets = [-nx, -1, 0, 1, nx]
print(f"{nx=} {ny=}")

nx=100 ny=100


# Build reaction diffusion matrix and test 

In [342]:
M = sp.diags(coeffs, [-nx, -1, 0, 1, nx], shape=(nx*ny, nx*ny), format='lil')
for j in range(y.size):
    for i in range(x.size):
        row = j*nx + i
        if i==0:
            M[row,row] += coeffs[1]
            if row-1 >= 0:
                M[row, row-1] = 0.0
        if i==nx-1:
            M[row, row] += coeffs[3]
            if row+1 < M.shape[1]:
                M[row, row+1] = 0.0
        if j==0:
            M[row, row] += coeffs[0]
        if j==ny-1:
            M[row, row] += coeffs[4]


In [343]:
# u = np.zeros(X.size)
deleteRows = sp.eye(M.shape[0]).tolil()
# for j, yj in enumerate(y):
#     for i, xi in enumerate(x):
#         if (xi-0.5)**2 + (yj-0.5)**2 < 0.2**2:
#             row = i + j*nx
#             deleteRows[row, row] = 0.0
#             u[row]= 1.0
u = np.zeros_like(X)
i0, i1 = int(7*ny/16), int(9*ny/16)
j0, j1 = int(4*nx/16), int(12*nx/16)
u[i0:i1, j0:j1] = 1.0
u = u.reshape((-1,))
for i in np.nonzero(u):
    deleteRows[i,i] = 0

print(f"{u.mean()=}, {u.min()=}, {u.max()=} {u.sum()=}")
fig, ax = plt.subplots()
c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
fig.colorbar(c, ax=ax)
plt.show()

u.mean()=0.065, u.min()=0.0, u.max()=1.0 u.sum()=650.0


In [344]:
M = deleteRows.dot(M)
A = -dt*M + sp.eye(M.shape[0])
A = A.tocsr()
for k in range(10):
    u = spl.spsolve(A, u)
    print(f"{u.mean()=}, {u.min()=}, {u.max()=}")

u.mean()=0.10035180036523843, u.min()=9.384132943638537e-08, u.max()=1.0
u.mean()=0.12156303252213109, u.min()=8.614394837433033e-07, u.max()=1.0
u.mean()=0.13800288503791003, u.min()=4.197591506584205e-06, u.max()=1.0
u.mean()=0.1519657828745761, u.min()=1.4464816939366313e-05, u.max()=1.0
u.mean()=0.1643494059734823, u.min()=3.961986536508898e-05, u.max()=1.0
u.mean()=0.17560817923773422, u.min()=9.191301155998336e-05, u.max()=1.0
u.mean()=0.18600910681546665, u.min()=0.00018790565432600886, u.max()=1.0
u.mean()=0.19572478529602008, u.min()=0.00034778291919670333, u.max()=1.0
u.mean()=0.20487442180069357, u.min()=0.0005941323337019349, u.max()=1.0
u.mean()=0.21354445038339526, u.min()=0.0009504550690336527, u.max()=1.0


In [345]:
fig, ax = plt.subplots()
c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
fig.colorbar(c, ax=ax)
# circ = Circle((0.5,0.5), 0.2, fill=False, linestyle='--', linewidth=3)
# ax.add_patch(circ)
plt.show()

# Build convection matrix

In [346]:
# Corners of the vessel
i0, i1 = int(7*ny/16), int(9*ny/16)
j0, j1 = int(4*nx/16), int(12*nx/16)

In [347]:
nPts = j1-j0
nVol = nx*ny
C = sp.diags([1/dx,-1/dx], [0,1], shape=(nPts, nPts+nVol), format='lil')
C[nPts-1,nPts] = 0
C = C.tocsr() 
# Add empty rows at the bottom of C to make it the right size
indptr = C.indptr.tolist()
for i in range(nVol):
    indptr.append(indptr[-1])
C = sp.csr_matrix((C.data, C.indices, indptr), shape=(nPts+nVol, nPts+nVol))

## Build matrix equating vascular voxel concentrations to nodal concentration

In [365]:
C4 = sp.lil_matrix((nVol, nPts))
I  = np.zeros(nVol)
for node,j in enumerate(range(j0, j1)):
    for i in range(i0, i1):
        row = i*nx+j
        C4[row,node] = 1
        I[row] = 1
I = sp.diags([I], [0]).tocsr()

# Check the loss
cv = np.ones(nPts) # Vascular nodal oxygen
print(np.any(I.dot(u)-C4.dot(cv)!=0.0)) # Return False if each voxel's oxygen is equal to its corresponding node's oxygen 

False


## Build mass exchange between vessels and tissue

In [364]:
NodesToEndothelialCells = sp.lil_matrix((nPts, nPts+nVol))
for node, j in enumerate(j0, j1):
    NodesToEndothelialCells[node, node] = -1
    for i in [i0,i1]:
        row = i*nx + j
        NodesToEndothelialCells[]

        

1.0
